In [3]:
import csv, sys

INPUT_ROWS_LIMIT= 100
FILENAME = 'dublinbikes_2020_Q1.csv'

In [4]:
with open(FILENAME, newline='') as f:
    reader = csv.reader(f)
    try:
        for row_i, row in enumerate(reader):
            if row_i >= INPUT_ROWS_LIMIT:
                break
            print(row)
    except csv.Error as e:
        sys.exit('file {}, line {}: {}'.format(filename, reader.line_num, e))

['STATION ID', 'TIME', 'LAST UPDATED', 'NAME', 'BIKE STANDS', 'AVAILABLE BIKE STANDS', 'AVAILABLE BIKES', 'STATUS', 'ADDRESS', 'LATITUDE', 'LONGITUDE']
['2', '2020-01-01 06:25:02', '2020-01-01 06:24:06', 'BLESSINGTON STREET', '20', '18', '2', 'Open', 'Blessington Street', '53.35677', '-6.26814']
['2', '2020-01-01 06:30:02', '2020-01-01 06:24:06', 'BLESSINGTON STREET', '20', '18', '2', 'Open', 'Blessington Street', '53.35677', '-6.26814']
['2', '2020-01-01 06:35:02', '2020-01-01 06:34:15', 'BLESSINGTON STREET', '20', '18', '2', 'Open', 'Blessington Street', '53.35677', '-6.26814']
['2', '2020-01-01 06:40:03', '2020-01-01 06:37:22', 'BLESSINGTON STREET', '20', '20', '0', 'Open', 'Blessington Street', '53.35677', '-6.26814']
['2', '2020-01-01 06:45:02', '2020-01-01 06:37:22', 'BLESSINGTON STREET', '20', '20', '0', 'Open', 'Blessington Street', '53.35677', '-6.26814']
['2', '2020-01-01 06:50:02', '2020-01-01 06:47:31', 'BLESSINGTON STREET', '20', '20', '0', 'Open', 'Blessington Street', '5